**importing packages**

In [19]:
import fiftyone as fo
from glob import glob
import os
from tqdm.notebook import tqdm

**Dawnloading data**

In [2]:
dataset = fo.zoo.load_zoo_dataset(
              "open-images-v7",
              split="train",
              label_types=["segmentations"],
              classes=["Car"],
              max_samples=15000,
          )
print('data downloaded')

data downloaded


Loading dataset

In [46]:
# The directory containing the dataset to import
dataset_dir = r"C:\Users\Tarek\fiftyone\open-images-v7\train"

# The type of the dataset being imported
dataset_type = fo.types.dataset_types.OpenImagesV7Dataset  # for example

# Import the dataset
dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    label_types=["segmentations"]
)

 100% |█████████████| 15003/15003 [26.7m elapsed, 0s remaining, 7.3 samples/s]      


**getting labels for downloaded images**

In [4]:
def extract_id_from_image_path(path):
    id = path.split('\\')[-1][0:16]
    return id
def extract_id_from_mask(path):
    mask_id  = path.split('\\')[-1]
    return mask_id

In [3]:
ROOT_DIR = r'C:\Users\Tarek\fiftyone\open-images-v7\train'
images_path = glob(os.path.join(ROOT_DIR,'data')+'/*')
#images_path = images_path[40:60]
image_ids = [extract_id_from_image_path(path) for path in images_path]

In [4]:
label_folders = glob(os.path.join(ROOT_DIR,'labels','masks')+'/*')
labels_paths = []
i = 1
for folder in tqdm(label_folders, desc='printing folders'):
    print(f'working on the  {i}th folder and folder path is :{folder}')
    labels = glob(folder+'/*')
    labels_filtred = [path for path in labels if extract_id_from_image_path(path) in image_ids and 'm0k4j' in path ]
    labels_paths.extend(labels_filtred)
    i+=1
print('labels extracted successfully')

printing folders:   0%|          | 0/16 [00:00<?, ?it/s]

working on the  1th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\0
working on the  2th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\1
working on the  3th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\2
working on the  4th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\3
working on the  5th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\4
working on the  6th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\5
working on the  7th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\6
working on the  8th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\7
working on the  9th folder and folder path is :C:\Users\Tarek\fiftyone\open-images-v7\train\labels\masks\8
working on the  10th folder and folde

In [11]:
labels_id = [extract_id_from_image_path(path) for path in labels_paths]

In [12]:
from collections import Counter


# Count the occurrences of each element
element_counts = Counter(labels_id)

# Print the counts
print(element_counts)


Counter({'0a564b751d8b8f2a': 26, '110925c163a316b2': 15, '268b4a7226895591': 15, '287ce6c11a1eef88': 15, '0d8429f7d260bd72': 14, '0e51b2d876346597': 14, '14722bbffd81481e': 14, '2aa1806cc9271721': 14, '0db3cf07f23eac39': 13, '277b830cab2b8084': 13, '281c7cfd998d8ab4': 13, '0051b190cdc3ab8a': 12, '02910b9f8334f999': 12, '032552f19b11d869': 12, '09c9d574b98bf57c': 12, '14240002411c8968': 12, '290e1233bb8c6009': 12, '31f8712e314e84a4': 12, '0722687af9cec71e': 11, '07e35a49413ccc50': 11, '09627060099f0749': 11, '0ad303897c23672e': 11, '0c7b12ac1c30c2ad': 11, '0cb3c55b5471442b': 11, '12681af5e1252faa': 11, '13fb75d7be21b566': 11, '16779561f37087cb': 11, '16b3597144a8032e': 11, '17f07bd286f81a99': 11, '19372bd5297d49cc': 11, '198a1cec36400786': 11, '1f0a947f4205c86c': 11, '20d8630e5828661a': 11, '20d86e67ea95c88c': 11, '252ed3f25a851ff4': 11, '2559b6deb6a6be77': 11, '263d93878361e028': 11, '27b3863d891d2622': 11, '27ec70fa648a389a': 11, '2fd006976b3988d9': 11, '309480b72990b726': 11, '31c436

In [12]:
import os
import shutil
def copy_files(file_paths, destination_directory):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
        
    for file_path in tqdm(file_paths, desc='copying files'):
        if os.path.isfile(file_path):
            try:
                shutil.copy(file_path, destination_directory)
                #print(f"Copied {file_path} to {destination_directory}")
            except Exception as e:
                print(f"Failed to copy {file_path}: {e}")
        else:
            print(f"{file_path} does not exist or is not a file.")

**converting from binary masks to coco format**

In [25]:
import datetime
import os
import numpy as np
import json
from PIL import Image
from tqdm.notebook import tqdm
from pycococreator.pycococreatortools import pycococreatortools

INFO = {
    "description": "Car_Tracking_2024",
    "url": "None",
    "version": "1.0",
    "year": 2024,
    "contributor": "Tarek",
    "date_created": datetime.datetime.now(datetime.UTC).isoformat(' ')
}

LICENSES = [{"id": 1, "name": "Zewail"}]
CATEGORIES = [{'id': 1, 'name': 'm0k4j', 'supercategory': 'none'}]

In [26]:
coco_output = {
    "info": INFO,
    "licenses": LICENSES,
    "categories": CATEGORIES,
    "images": [],
    "annotations": []
}


In [21]:
image_paths = images_path
annotation_files = labels_paths

In [22]:
print(f'the number of images is: {len(image_paths)}')
print(f'the number of masks is: {len(annotation_files)}')

the number of images is: 15003
the number of masks is: 32532


In [23]:
image_id = 1
for image_path in tqdm(image_paths, desc='images preprocessing'):
    image = Image.open(image_path)
    image_filename = image_path.split('\\')[-1]
    image_info = pycococreatortools.create_image_info(image_id, os.path.basename(image_filename), image.size)
    coco_output["images"].append(image_info)
    image_id += 1

images preprocessing:   0%|          | 0/15003 [00:00<?, ?it/s]

In [ ]:
from PIL import Image

segmentation_id = 1
for annotation_filepath in tqdm(annotation_files, desc='labels preprocessing'):
    image_id = [f for f in coco_output['images'] if f['file_name'][0:16] == extract_id_from_image_path(annotation_filepath)][0]['id']
    annotation_filename = annotation_filepath.split('\\')[-1]
    class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]
    category_info = {'id': class_id, 'is_crowd': 'crowd' in annotation_filename}

    # Load and check mask size
    mask_image = Image.open(annotation_filepath).convert('1')
    
    # Resize mask if needed
    if mask_image.size != (coco_output['images'][image_id-1]['width'], coco_output['images'][image_id-1]['height']):
        #print(f"Resizing mask from {mask_image.size} to {(coco_output['images'][image_id-1]['width'], coco_output['images'][image_id-1]['height'])}")
        mask_image = mask_image.resize((coco_output['images'][image_id-1]['width'], coco_output['images'][image_id-1]['height']), Image.NEAREST)
    
    binary_mask = np.asarray(mask_image).astype(np.uint8)

    # Debugging: Print sizes to ensure they match
    #print(f"Processing image ID: {image_id}, Image size: {coco_output['images'][image_id-1]['width']}x{coco_output['images'][image_id-1]['height']}, Mask size: {mask_image.size}")

    annotation_info = pycococreatortools.create_annotation_info(segmentation_id, image_id, category_info, binary_mask, mask_image.size, tolerance=2)

    if annotation_info is not None:
        coco_output["annotations"].append(annotation_info)
        segmentation_id += 1


In [76]:
coco_output['categories'] = [{'id': 1, 'name': 'car', 'supercategory': 'none'}]

import json
with open('data.json', 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [2]:
import json

# Specify the path to your JSON file
json_file_path = 'test/data.json'

# Open and load the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)  # Save the JSON content to a Python variable

**From COCO to YOLO**

In [28]:
import json
import os
import shutil
import yaml
def convert_to_yolo(input_images_path, input_json_path, output_labels_path):
    # Open JSON file containing image annotations
    f = open(input_json_path)
    data = json.load(f)
    f.close()

    # Create directories for output images and labels
    #os.makedirs(output_images_path, exist_ok=True)
    os.makedirs(output_labels_path, exist_ok=True)

    # List to store filenames
    file_names = []
    for filename in os.listdir(input_images_path):
        if filename.endswith(".jpg"):
            #source = os.path.join(input_images_path, filename)
            #destination = os.path.join(output_images_path, filename)
            #shutil.copy(source, destination)
            file_names.append(filename)

    # Function to get image annotations
    def get_img_ann(image_id):
        return [ann for ann in data['annotations'] if ann['image_id'] == image_id]

    # Function to get image data
    def get_img(filename):
        return next((img for img in data['images'] if img['file_name'] == filename), None)

    # Iterate through filenames and process each image
    for filename in tqdm(file_names, desc='process each image'):
        img = get_img(filename)
        img_id = img['id']
        img_w = img['width']
        img_h = img['height']
        img_ann = get_img_ann(img_id)

        # Write normalized polygon data to a text file
        if img_ann:
            with open(os.path.join(output_labels_path, f"{os.path.splitext(filename)[0]}.txt"), "a") as file_object:
                for ann in img_ann:
                    current_category = ann['category_id'] - 1
                    polygon = ann['segmentation'][0]
                    normalized_polygon = [format(coord / img_w if i % 2 == 0 else coord / img_h, '.6f') for i, coord in enumerate(polygon)]
                    file_object.write(f"{current_category} " + " ".join(normalized_polygon) + "\n")

In [29]:
input_image_paths = r'C:\Users\Tarek\fiftyone\open-images-v7\train\data'
input_json_path = r'test\data.json'
output_labels_path = r'Labels'
convert_to_yolo(input_image_paths, input_json_path, output_labels_path)

process each image:   0%|          | 0/15003 [00:00<?, ?it/s]

In [ ]:
# https://youtu.be/NYeJvxe5nYw
"""
This code defines a function to display an image with its corresponding annotations. 
It reads an image and its associated annotation file in the YOLO v8 text format, 
then plots the image along with colored polygons representing the annotated regions. 
The polygons are drawn according to the coordinates provided in the annotation file, 
and colors are assigned based on the category ID.

"""

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

def display_image_with_annotations(image_path, annotation_path, colors=None):
    # Load image using OpenCV and convert it from BGR to RGB color space
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = image.shape
    
    # Create a figure and axis to display the image
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    ax.axis('off')  # Turn off the axes

    # Define a default color map if none is provided
    if colors is None:
        colors = plt.cm.get_cmap('tab10')

    # Open the annotation file and process each line
    with open(annotation_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            category_id = int(parts[0])
            # Choose color based on category ID, looping through color map if more than 10 categories
            color = colors(category_id % 10)
            # Extract normalized polygon coordinates and denormalize them
            polygon = [float(coord) for coord in parts[1:]]
            polygon = [coord * img_w if i % 2 == 0 else coord * img_h for i, coord in enumerate(polygon)]
            # Reshape into (num_points, 2) array
            polygon = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
            # Create a Polygon patch using the denormalized coordinates
            patch = patches.Polygon(polygon, closed=True, edgecolor=color, fill=False)
            # Add the patch to the plot to display the annotated region
            ax.add_patch(patch)

    plt.show()  # Display the image with annotations

# Example usage with specified image and annotation paths
image_path = "C:\Users\Tarek\fiftyone\open-images-v7\train\data\00cac7ea145fc734.jpg"
annotation_path = "Labels\00cac7ea145fc734.txt"
display_image_with_annotations(image_path, annotation_path)

**Splittin Dataset**

In [20]:
all_images_path = glob(r'C:\Users\Tarek\fiftyone\open-images-v7\train\data' +'/*')
all_labels_path = glob(r'D:\self-grinding\deeplearning\Project\object-tracking\Labels' +'/*')
print(len(all_images_path))
print(len(all_labels_path))

15003
15003


In [21]:
all_ids = [extract_id_from_image_path(path) for path in all_images_path]

In [22]:
from sklearn.model_selection import train_test_split
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, shuffle=True)

In [23]:
images_root_dir = r'C:\Users\Tarek\fiftyone\open-images-v7\train\data'
labels_root_dir = r'Labels'

In [24]:
train_image_paths = [os.path.join(images_root_dir,f'{id}.jpg') for id in train_ids]
val_image_paths = [os.path.join(images_root_dir,f'{id}.jpg') for id in val_ids]
train_label_paths = [os.path.join(labels_root_dir,f'{id}.txt') for id in train_ids]
val_label_paths = [os.path.join(labels_root_dir,f'{id}.txt') for id in val_ids]

In [25]:
copy_files(train_image_paths, r'data\images\train')
copy_files(val_image_paths, r'data\images\val' )
copy_files(train_label_paths, r'data\labels\train')
copy_files(val_label_paths, r'data\labels\val' )

copying files:   0%|          | 0/12002 [00:00<?, ?it/s]

copying files:   0%|          | 0/3001 [00:00<?, ?it/s]

copying files:   0%|          | 0/12002 [00:00<?, ?it/s]

copying files:   0%|          | 0/3001 [00:00<?, ?it/s]